In [1]:
from audiolm_pytorch import SoundStream, SoundStreamTrainer
soundstream = SoundStream(
    codebook_size = 1024,
    rq_num_quantizers = 8,
)
soundstream.load("content/soundstream/results3/soundstream.9000.pt")
soundstream.to("cuda")



2023-01-04 17:23:03 | INFO | fairseq.tasks.text_to_speech | Please install tensorboardX: pip install tensorboardX
2023-01-04 17:23:03.874996: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-04 17:23:04.390471: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-01-04 17:23:04.390521: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-01-04 17:23:04.390525: W tensorflow/compiler/tf

SoundStream(
  (encoder): Sequential(
    (0): CausalConv1d(
      (conv): Conv1d(1, 32, kernel_size=(7,), stride=(1,))
    )
    (1): Sequential(
      (0): Residual(
        (fn): Sequential(
          (0): CausalConv1d(
            (conv): Conv1d(32, 32, kernel_size=(7,), stride=(1,))
          )
          (1): ELU(alpha=1.0)
          (2): CausalConv1d(
            (conv): Conv1d(32, 32, kernel_size=(1,), stride=(1,))
          )
          (3): ELU(alpha=1.0)
        )
      )
      (1): Residual(
        (fn): Sequential(
          (0): CausalConv1d(
            (conv): Conv1d(32, 32, kernel_size=(7,), stride=(1,), dilation=(3,))
          )
          (1): ELU(alpha=1.0)
          (2): CausalConv1d(
            (conv): Conv1d(32, 32, kernel_size=(1,), stride=(1,))
          )
          (3): ELU(alpha=1.0)
        )
      )
      (2): Residual(
        (fn): Sequential(
          (0): CausalConv1d(
            (conv): Conv1d(32, 32, kernel_size=(7,), stride=(1,), dilation=(9,))
   

In [19]:
import sys
from importlib import reload
import network_models.soundstream_lstm.LSTM_dataset as lds
#from network_models.soundstream_lstm.LSTM_dataset  import AudioEmotionTessDataset
reload(lds)


tess_dataset = lds.AudioEmotionTessDataset("../tess")
encoded_dataset = lds.AudioEmotionTessSoundStreamEncodedDataset(dataSet= tess_dataset, soundStream=soundstream)

In [29]:
import torch
from network_models.soundstream_lstm.lstm_model import LSTM1
reload(LSTM1)
# print(encoded_dataset.__getitem__(3).shape)
model = LSTM1(encoded_dataset.__len__(), 512, 4, 4, 0)
print(torch.tensor(encoded_dataset.__getitem__(3)[0]).shape)
model(torch.tensor(encoded_dataset.__getitem__(3)[0]))

ImportError: module LSTM1 not in sys.modules